## Importing dependencies

In [1]:
import os
import joblib
from azureml.core import Workspace, Experiment
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.train.automl import AutoMLConfig

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd


## Initialize workspace

In [2]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

quick-starts-ws-141564
aml-quickstarts-141564
southcentralus
d7f39349-a66b-446e-aba6-0053c2cf1c11


## Auto ML Experiment

In [3]:
# Choose a name for the run history container in the workspace.
# NOTE: update these to match your existing experiment name
experiment_name = 'data_quality_analysis'
project_folder = './catalogue-scorecard-project'

experiment = Experiment(ws, experiment_name)
experiment

Name,Workspace,Report Page,Docs Page
data_quality_analysis,quick-starts-ws-141564,Link to Azure Machine Learning studio,Link to Documentation


## Create an AmlCompute cluster

In [4]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# Choose a name for your CPU cluster
amlcompute_cluster_name = "covcompute"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D12_V2',#for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)

Found existing cluster, use it.

Running


## Data import

In [5]:
import pandas as pd

auto_ds_path="https://raw.githubusercontent.com/shatakshipachori/data_quality_analysis/main/catalogue-scorecard.csv"
ds=TabularDatasetFactory.from_delimited_files(path=auto_ds_path)


## AutoML Configuration

In [15]:
import logging
import time

# TODO: Put your automl settings here
automl_settings = {
    "n_cross_validations":2,
    "primary_metric":'normalized_root_mean_squared_error',
    "enable_early_stopping":True,
    "experiment_timeout_minutes":15,
    "max_concurrent_iterations":5,
   }

# TODO: Put your automl config here
automl_config = AutoMLConfig(
    task='regression',
    compute_target='covcompute',
    training_data=ds,
    label_column_name='score',
    **automl_settings
    )

## Submit the experiment

In [16]:
remote_run = experiment.submit(automl_config, show_output=True)

Running on remote.
No run_configuration provided, running on ubicompute with default configuration
Running on remote compute: ubicompute
Parent Run ID: AutoML_5b758294-4017-4f7b-ae9f-06b914d652ac

Something went wrong while printing the experiment progress but the run is still executing on the compute target. 
Please check portal for updated status: https://ml.azure.com/experiments/country_vaccinations/runs/AutoML_5b758294-4017-4f7b-ae9f-06b914d652ac?wsid=/subscriptions/d7f39349-a66b-446e-aba6-0053c2cf1c11/resourcegroups/aml-quickstarts-141564/workspaces/quick-starts-ws-141564


## Run Details

In [17]:
from azureml.widgets import RunDetails
RunDetails(remote_run).show()
remote_run.wait_for_completion(show_output=True)

## Best Model

In [ ]:
# Retrieve and save your best automl model.
best_remote_run,best_automl_model=remote_run.get_output()
best_run_metrics=remote_run.get_metrics()
for primary_metric in best_run_metrics:
    metric=best_run_metrics[primary_metric]
    print(primary_metric,metric)

In [ ]:
print(best_remote_run)
best_remote_run